In [40]:
import numpy as np
import pandas as pd
from importlib import reload
import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import shared
import config
import provider_yfinance as provider
import model
import plot

reload(shared)
reload(config)
reload(provider)
reload(model)
reload(plot)

plot.Notebook()

mpl.rcParams['figure.figsize'] = (16, 9)
mpl.rcParams['axes.grid'] = False
mpl.rcParams['axes.unicode_minus']=False
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

cfg = config.get_config('^GDAXI')
mdl_cfg = config.get_config('^GDAXI')
# overwrite download_end_dt: use cached data
# config.save_config(cfg)

config> created config from file: './config.json'
config> config
        - base:
            - config_file_path: /mnt/c/notebooks/sandbox/config.json
        - datasets:
            - stocks: 30
            - benchmarks: 69
        - prepare:
            - data_start_dt: 2018-02-25
            - data_end_dt: 2020-01-26
            - cache_dir: /mnt/c/notebooks/sandbox/cache/20200126/
        - train:            
            - window_trading_days: [3, 5, 21, 35, 50]
            - lag_trading_days: [1, 2, 3, 4, 5]
            - label_max_high_weight: 3.0
            - label_max_close_weight: 1.0
            - settings: 5
        - model:
            - max_samples: 40
            - batch_size: 32
            - learning_rate: 0.1
            - learning_rate_decay: 0.75
            - lstm_hidden_size: 256
            - early_stopping_patience: 10
            - validation_monitor: val_mean_squared_error
            - max_epochs: 1000
            - base_dir: /mnt/c/notebooks/sandbox/model/202

In [41]:
%%time 
# 344 ms
cfg_stocks, data_stocks = provider.load_stocks(cfg)
cfg_benchmarks, data_benchmarks = provider.load_benchmarks(cfg)

CPU times: user 297 ms, sys: 62.5 ms, total: 359 ms
Wall time: 329 ms


In [9]:
%%time 
# total: 2.55 s
prep_stocks = provider.prepare_stocks(cfg, data_stocks)
prep_benchmarks = provider.prepare_benchmarks(cfg, data_benchmarks)

CPU times: user 344 ms, sys: 31.2 ms, total: 375 ms
Wall time: 378 ms


In [10]:
%%time
# total: 1.2 s
enc_stocks = provider.encode_stocks(cfg, prep_stocks)
enc_benchmarks = provider.encode_benchmarks(cfg, prep_benchmarks, prep_stocks)

CPU times: user 62.5 ms, sys: 62.5 ms, total: 125 ms
Wall time: 116 ms


In [7]:
submodel_settings = cfg.train.settings[-1]
submodel_settings

Munch({'ensemble_weight': 1.0, 'float_precision': 100.0, 'id': 'lookback_100-label_7', 'label_days': 7, 'lookback_days': 100, 'max_samples': 26, 'num_features': 1319, 'prev_year_samples_after': 12, 'prev_year_samples_before': 5, 'sample_manifolds': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 4, 5, 6, 7]})

In [33]:
ticker_name = '1COV.DE'
ticker_data = enc_stocks[ticker_name]
end_dt = shared.parse_datetime(cfg.prepare.download_end_dt)
ticker_dates = ticker_data.iloc[ticker_data.index <= end_dt].index
lookback_end_date = ticker_dates[-1]
lookback_start_date = ticker_dates[-(1 + submodel_settings.lookback_days)]
back_data = ticker_data.iloc[(ticker_data.index >= lookback_start_date) & (ticker_data.index <= lookback_end_date)]
print(f"{submodel_settings.id}> prediction lookback from '{lookback_start_date}' to '{lookback_end_date}' for the next {submodel_settings.label_days} days")

bm_X = provider.generate_relative_benchmark_data(cfg, submodel_settings, enc_benchmarks, lookback_start_date, lookback_end_date)
print(f"{submodel_settings.id}> benchmark data shape: {bm_X.shape}")
X = provider.generate_sample_x(cfg, submodel_settings, ticker_data, bm_X, lookback_start_date, lookback_end_date)
print(f"{submodel_settings.id}> X shape: {X.shape}")

lookback_100-label_7> prediction lookback from '2019-08-29 00:00:00' to '2020-01-24 00:00:00' for the next 7 days
lookback_100-label_7> benchmark data shape: (101, 1292)
lookback_100-label_7> X shape: (1, 100, 1319)


In [43]:
mdl = model.create_model(cfg, submodel_settings, ticker_name=ticker_name, train_mode=False, input_shape=X.shape)
prediction = mdl.predict(X)[0]
prediction

array([2.8960767], dtype=float32)

In [45]:
mdl0 = model.create_model(mdl_cfg, submodel_settings, train_mode=False, input_shape=X.shape)
prediction0 = mdl0.predict(X)[0]
prediction0

array([3.8358867], dtype=float32)